![Astrofisica Computacional](../logo.PNG)

---
## 01. Gravitational Waves Signal


Eduard Larrañaga (ealarranaga@unal.edu.co)

---


### About this notebook

In this notebook we present the method to In this notebook we will read and visualize one of the gravitational waves signal detected by LIGO.define a straight line that fits a dataset.

---